In [1]:
from __future__ import division
import time
from datetime import datetime as dt

import argparse
from itertools import permutations

import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from dgl.data import SBMMixture
import sbm
import gnn


In [2]:
batch_size = 1
gpu=-1
lr = 0.004
n_communities = 5
n_epochs = 1 
n_features = 8 
clip_grad_norm = 40.0 
verbose = True
interval = 10

In [3]:
n_graphs = 1 
p = 0 
q = 18 
n_layers = 30 
n_nodes = 10 
optim = 'Adamax' 
radius  = 2 

In [4]:
import dgl.function as fn

In [5]:
dev = th.device('cpu')

In [6]:
K = n_communities 

In [14]:
training_dataset = sbm.SBM(1, 10, 2, 0, 5) ## n_graphs, n_nodes, n_communities, p, q*n_nodes

In [15]:
g, lg, g_d, lg_d, pm_pd = training_dataset.__getitem__(0)

In [16]:
print(g.all_edges())
print(g_d)

(tensor([0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5, 5, 5, 7, 7, 8, 8, 8, 8, 9, 9]), tensor([5, 8, 5, 7, 9, 5, 7, 8, 8, 9, 5, 8, 0, 1, 2, 4, 1, 2, 0, 2, 3, 4, 1, 3]))
tensor([[2.],
        [3.],
        [3.],
        [2.],
        [2.],
        [4.],
        [0.],
        [2.],
        [4.],
        [2.]])


In [17]:
print(lg.all_edges())
print(lg_d)

(tensor([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  4,  5,  5,  5,  6,  7,  7,  7,
         8,  8,  8,  9, 10, 10, 10, 11, 11, 11, 12, 13, 13, 14, 14, 15, 16, 16,
        17, 17, 18, 19, 19, 20, 21, 22, 22, 23]), tensor([13, 14, 15, 19, 20, 21, 12, 14, 15, 17, 23, 12, 13, 15, 16, 18, 20, 21,
        18, 19, 21, 22, 12, 13, 14, 18, 19, 20,  1,  3,  4,  6,  7, 11,  2,  4,
         5,  7,  0,  5,  6,  9, 10,  2,  3,  8]))
tensor([[3.],
        [3.],
        [3.],
        [1.],
        [1.],
        [3.],
        [1.],
        [3.],
        [3.],
        [1.],
        [3.],
        [3.],
        [1.],
        [2.],
        [2.],
        [1.],
        [2.],
        [2.],
        [1.],
        [2.],
        [1.],
        [1.],
        [2.],
        [1.]])


In [18]:
print(lg.__len__())

24


In [19]:
print(pm_pd)

tensor([0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5, 5, 5, 7, 7, 8, 8, 8, 8, 9, 9])


In [20]:
model = GNN([1, 8, 8, 8, 8], 2, 2).to(dev)

In [21]:
aggregate_init(g)
aggregate_init(lg)
model(g, lg, g_d, lg_d, pm_pd)

/anaconda3/envs/py36/lib/python3.6/site-packages/dgl/frame.py:204: UserWarning: Initializer is not set. Use zero initializer instead. To suppress this warning, use `set_initializer` to explicitly specify which initializer to use.
  dgl_warning('Initializer is not set. Use zero initializer instead.'


tensor([[ 0.0764,  0.1488],
        [-0.3616, -0.1104],
        [-0.1766,  0.0534],
        [-0.6173,  0.0932],
        [ 0.0764,  0.1488],
        [ 0.3233, -0.3750],
        [-0.6126, -0.2924],
        [ 0.1342, -0.4919],
        [ 0.1266, -0.9273],
        [-0.4417, -1.1652]], grad_fn=<AddmmBackward>)

In [13]:
ph = 1000

def pp(l):
    for i in range(len(l)):
        print(l[i])

def printnode(g):
    for i in range(g.number_of_nodes()):
        print(g.nodes[i].data['id'])
        print(g.nodes[i].data['z'])
        print(g.nodes[i].data['t'])
    #print(node.data['zz'])

def id_to_m(edges):
    return {'m': edges.src['id']}

def m_to_t(nodes):
    #print(nodes.__len__())
    #print(nodes.nodes())
    new_t_list = []
    for i in range(nodes.__len__()):
        cur = nodes.data['t'][i].tolist()
        
        if not isinstance(cur, list):
            cur = [cur]
            
        new_t = cur[:cur.index(-1)]
        
        for m in nodes.mailbox['m'][i]:
            m = m.tolist()
            if isinstance(m, list):
                if -1 in m:
                    new_t = new_t + m[:m.index(-1)]
                else:
                    new_t = new_t + m
            else:
                new_t.append(m)
        
        new_t = list(set(new_t))
        
        if len(new_t) < ph:
            new_t = new_t + [-1] * (ph-len(new_t))
        
        new_t_list.append(new_t)
        
    #print(th.tensor(new_t_list))
    return {'t': th.tensor(new_t_list)}

def m_to_tt(nodes):
    #print(nodes.__len__())
    #print(nodes.nodes())
    new_t_list = []
    for i in range(nodes.__len__()):
        cur = nodes.data['t'][i].tolist()
        
        if not isinstance(cur, list):
            cur = [cur]
            
        new_t = cur[:cur.index(-1)]
        
        for m in nodes.mailbox['m'][i]:
            m = m.tolist()
            if isinstance(m, list):
                if -1 in m:
                    new_t = new_t + m[:m.index(-1)]
                else:
                    new_t = new_t + m
            else:
                new_t.append(m)
        
        new_t = list(set(new_t))
        
        if len(new_t) < ph:
            new_t = new_t + [-1] * (ph-len(new_t))
        
        new_t_list.append(new_t)
        
    #print(th.tensor(new_t_list))
    return {'tt': th.tensor(new_t_list)}

def t_to_m(edges):
    return {'m': edges.src['t']}

def aggregate_init(g):
    for i in range(g.number_of_nodes()):
        g.nodes[i].data['id'] = th.tensor([i])

    g.ndata['t'] = th.ones([g.number_of_nodes(), ph], dtype=th.int64) * -1
    g.ndata['tt'] = th.ones([g.number_of_nodes(), ph], dtype=th.int64) * -1

    g.register_message_func(id_to_m)
    g.register_reduce_func(m_to_t)

    g.send(g.edges())
    g.recv(g.nodes())

    g.register_message_func(t_to_m)
    g.register_reduce_func(m_to_tt)

    g.send(g.edges())
    g.recv(g.nodes())

class GNNModule(nn.Module):
    def __init__(self, in_feats, out_feats, radius):
        super().__init__()
        self.in_feats = in_feats
        self.out_feats = out_feats
        self.radius = radius

        new_linear = lambda: nn.Linear(in_feats, out_feats)
        new_linear_list = lambda: nn.ModuleList([new_linear() for i in range(radius)])

        self.theta_x, self.theta_deg, self.theta_y = \
            new_linear(), new_linear(), new_linear()
        self.theta_list = new_linear_list()

        self.gamma_y, self.gamma_deg, self.gamma_x = \
            new_linear(), new_linear(), new_linear()
        self.gamma_list = new_linear_list()

        self.bn_x = nn.BatchNorm1d(out_feats)
        self.bn_y = nn.BatchNorm1d(out_feats)

    
    def t_to_feature(self, g):
        #print(g.ndata['z'])
        for i in range(g.number_of_nodes()):
            ind = g.nodes[i].data['t'][0].tolist()
            
            if ind == [0] * ph or ind[0] == -1:
                ind = [i]
            elif -1 in ind:
                ind = ind[:ind.index(-1)]
                
            ind = th.tensor(ind)
            #print("================  t_to_feature ============")
            #print(g.ndata['z'])
            #print(ind)
            #print((g.nodes[ind].data['z']))
            #print(th.sum(g.nodes[ind].data['z'], dim=0))
            #print(th.tensor([th.sum(g.nodes[ind].data['z'], dim=0).tolist()]))
            if self.in_feats == 1:
                g.nodes[i].data['zz'] = th.sum(g.nodes[ind].data['z'], dim=0)
            else:
                g.nodes[i].data['zz'] = th.tensor([th.sum(g.nodes[ind].data['z'], dim=0).tolist()])
        
            
    def tt_to_feature(self, g):
        #print(g.ndata['z'])
        for i in range(g.number_of_nodes()):
            ind = g.nodes[i].data['tt'][0].tolist()
            
            if ind == [0] * ph or ind[0] == -1:
                ind = [i]
            elif -1 in ind:
                ind = ind[:ind.index(-1)]
                
            ind = th.tensor(ind)
            #print("================  t_to_feature ============")
            #print(g.ndata['z'])
            #print(ind)
            #print((g.nodes[ind].data['z']))
            #print(th.sum(g.nodes[ind].data['z'], dim=0))
            #print(th.tensor([th.sum(g.nodes[ind].data['z'], dim=0).tolist()]))
            if self.in_feats == 1:
                g.nodes[i].data['zz'] = th.sum(g.nodes[ind].data['z'], dim=0)
            else:
                g.nodes[i].data['zz'] = th.tensor([th.sum(g.nodes[ind].data['z'], dim=0).tolist()])

    def aggregate(self, g, z):
        ###g.register_message_func(id_to_m)
        
        ###g.register_reduce_func(m_to_t)
        
        z_list = []
        g.set_n_repr({'z' : z})
        
        ###self.aggregate_init(g)
        #g.update_all(fn.copy_src(src='id', out='m'), fn.sum(msg='m', out='z'))
        ###g.send(g.edges())
        
        ###g.recv(g.nodes())
        
        ###g.register_message_func(t_to_m)
        
        self.t_to_feature(g)
        
        if self.in_feats == 1:
            z = th.tensor([[b] for b in g.ndata.pop('zz').tolist()])
        else:
            z = g.ndata.pop('zz')
        
        z_list.append(z)
        '''
        for i in range(self.radius-1):
           for j in range(2 ** i):
                g.send(g.edges())
                g.recv(g.nodes())
                #g.update_all(fn.copy_src(src='z', out='m'), fn.sum(msg='m', out='z'))
        '''    
        self.tt_to_feature(g)
            
        if self.in_feats == 1:
            z = th.tensor([[b] for b in g.ndata.pop('zz').tolist()])
        else:
            z = g.ndata.pop('zz')
            
            #print('============== z ==============')
            #print(z)
            
        z_list.append(z)
        
        #print('==============. g  =========')
        #printnode(g)
        #print('==============. zlist  =========')
        #print(z_list)
        return z_list

    def forward(self, g, lg, x, y, deg_g, deg_lg, pm_pd):
        pmpd_x = F.embedding(pm_pd, x)
        
        ##print(pmpd_x)
        
        a = self.aggregate(g,x)
        
        ##print(a)
        
        sum_x = sum(theta(z) for theta, z in zip(self.theta_list, a))

        g.set_e_repr({'y' : y})
        g.update_all(fn.copy_edge(edge='y', out='m'), fn.sum('m', 'pmpd_y'))
        pmpd_y = g.pop_n_repr('pmpd_y')

        x = self.theta_x(x) + self.theta_deg(deg_g * x) + sum_x + self.theta_y(pmpd_y)
        n = self.out_feats // 2
        x = th.cat([x[:, :n], F.relu(x[:, n:])], 1)
        x = self.bn_x(x)

        sum_y = sum(gamma(z) for gamma, z in zip(self.gamma_list, self.aggregate(lg, y)))

        y = self.gamma_y(y) + self.gamma_deg(deg_lg * y) + sum_y + self.gamma_x(pmpd_x)
        y = th.cat([y[:, :n], F.relu(y[:, n:])], 1)
        y = self.bn_y(y)

        return x, y

class GNN(nn.Module):
    def __init__(self, feats, radius, n_classes):
        """
        Parameters
        ----------
        g : networkx.DiGraph
        """
        super(GNN, self).__init__()
        self.linear = nn.Linear(feats[-1], n_classes)
        self.module_list = nn.ModuleList([GNNModule(m, n, radius)
                                          for m, n in zip(feats[:-1], feats[1:])])

    def forward(self, g, lg, deg_g, deg_lg, pm_pd):
        x, y = deg_g, deg_lg
        for module in self.module_list:
            x, y = module(g, lg, x, y, deg_g, deg_lg, pm_pd)
        return self.linear(x)

In [ ]:


##training_dataset = sbm.SBM(args.n_graphs, args.n_nodes, K, p, q)
##training_dataset = SBMMixture(args.n_graphs, args.n_nodes, K)
##training_loader = DataLoader(training_dataset, args.batch_size, collate_fn=training_dataset.collate_fn, drop_last=True)

ones = th.ones(args.n_nodes // K)
y_list = [th.cat([x * ones for x in p]).long().to(dev)
          for p in permutations(range(K))]

##feats = [1] + [args.n_features] * args.n_layers + [K]
feats = [1] + [args.n_features] * args.n_layers
model = gnn.GNN(feats, args.radius, K).to(dev)
optimizer = getattr(optim, args.optim)(model.parameters(), lr=args.lr)


def compute_overlap(z_list):
    ybar_list = [th.max(z, 1)[1] for z in z_list]
    overlap_list = []
    for y_bar in ybar_list:
        accuracy = max(th.sum(y_bar == y).item()
                       for y in y_list) / args.n_nodes
        overlap = (accuracy - 1 / K) / (1 - 1 / K)
        overlap_list.append(overlap)
    return sum(overlap_list) / len(overlap_list)


def from_np(f, *args):
    def wrap(*args):
        new = [th.from_numpy(x) if isinstance(
            x, np.ndarray) else x for x in args]
        return f(*new)
    return wrap


@from_np
def step(i, j, g, lg, deg_g, deg_lg, pm_pd):
    """ One step of training. """
    deg_g = deg_g.to(dev)
    deg_lg = deg_lg.to(dev)
    pm_pd = pm_pd.to(dev)
    t0 = time.time()
    z = model(g, lg, deg_g, deg_lg, pm_pd)
    t_forward = time.time() - t0

    z_list = th.chunk(z, args.batch_size, 0)
    loss = sum(min(F.cross_entropy(z, y) for y in y_list)
               for z in z_list) / args.batch_size
    overlap = compute_overlap(z_list)

    optimizer.zero_grad()
    t0 = time.time()
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad_norm)
    t_backward = time.time() - t0
    optimizer.step()

    return loss, overlap, t_forward, t_backward


@from_np
def inference(g, lg, deg_g, deg_lg, pm_pd):
    deg_g = deg_g.to(dev)
    deg_lg = deg_lg.to(dev)
    pm_pd = pm_pd.to(dev)

    z = model(g, lg, deg_g, deg_lg, pm_pd)

    return z


def test():
    p_list = [6, 5.5, 5, 4.5, 1.5, 1, 0.5, 0]
    q_list = [0, 0.5, 1, 1.5, 4.5, 5, 5.5, 6]
    N = 1
    overlap_list = []
    for p, q in zip(p_list, q_list):
        dataset = SBMMixture(N, args.n_nodes, K, pq=[[p, q]] * N)
        loader = DataLoader(dataset, N, collate_fn=dataset.collate_fn)
        g, lg, deg_g, deg_lg, pm_pd = next(iter(loader))
        z = inference(g, lg, deg_g, deg_lg, pm_pd)
        overlap_list.append(compute_overlap(th.chunk(z, N, 0)))
    return overlap_list


n_iterations = args.n_graphs // args.batch_size
for i in range(args.n_epochs):
    total_loss, total_overlap, s_forward, s_backward = 0, 0, 0, 0
# for j, [g, lg, deg_g, deg_lg, pm_pd] in enumerate(training_loader):
    for j in range(args.n_graphs):
        g, lg, deg_g, deg_lg, pm_pd = sbm.SBM(
            1, args.n_nodes, K, p, q).__getitem__(0)

        loss, overlap, t_forward, t_backward = step(
            i, j, g, lg, deg_g, deg_lg, pm_pd)

        total_loss += loss
        total_overlap += overlap
        s_forward += t_forward
        s_backward += t_backward

        if j % args.interval == 0:

            if j > 0:
                print('=================== interval %d | loss %0.3f | overlap %.3f ==================='
                      % (j, interval_loss/args.interval, interval_overlap/args.interval))

            interval_loss = 0
            interval_overlap = 0

        interval_loss += loss
        interval_overlap += overlap

        epoch = '0' * (len(str(args.n_epochs)) - len(str(i)))
        iteration = '0' * (len(str(n_iterations)) - len(str(j)))
        if args.verbose:
            print(dt.now(), '[epoch %s%d iteration %s%d]loss %.3f | overlap %.3f'
                  % (epoch, i, iteration, j, loss, overlap))

    epoch = '0' * (len(str(args.n_epochs)) - len(str(i)))
    loss = total_loss / (j + 1)
    overlap = total_overlap / (j + 1)
    t_forward = s_forward / (j + 1)
    t_backward = s_backward / (j + 1)
    print('[epoch %s%d]loss %.3f | overlap %.3f | forward time %.3fs | backward time %.3fs'
          % (epoch, i, loss, overlap, t_forward, t_backward))

    overlap_list = test()
    overlap_str = ' - '.join(['%.3f' % overlap for overlap in overlap_list])
    print('[epoch %s%d]overlap: %s' % (epoch, i, overlap_str))

th.save(model.state_dict(), args.save_path)